# Spotipy - Retrieving playlist and track data from Spotify

## 1. Workplace Preparation

On Spotify's Developer website, a python library called **Spotipy** is recommended. I decided to utilise this library for my data collection from Spotify.

Library Source/Documentation: https://github.com/plamere/spotipy

In [4]:
# Imports the library for utilising Spotify's API by Python
!pip install spotipy


         .:::.     .::.       
        ....yy:    .yy.       
        :.  .yy.    y.        
             :y:   .:         
             .yy  .:          
              yy..:           
              :y:.            
              .y.             
             .:.              
        ....:.                
        :::.                  

• Project files and data should be stored in /project. This is shared among everyone
  in the project.
• Personal files and configuration should be stored in /home/faculty.
• Files outside /project and /home/faculty will be lost when this server is terminated.
• Create custom environments to setup your servers reproducibly.



In [5]:
# Token initialisation
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id="7b1fa7a7eb25461f8d3a4a66e1966de5",
                                                           client_secret="cd8fa0032d964a1bbf6381b3b471d74e"))


In [29]:
import sys
import pprint
from pprint import pprint

## 2. Which Playlist?

First of all, I want to retrieve a list of featured playlists (Editor's Picks) on Spotify to see if there is a playlist that looks like it is worthy of investigation. These playlists usually contain popular tracks and are curated by Spotify's in-house team (Occhino, 2020). 

In [25]:
response = sp.featured_playlists()
print(response['message'])

while response:
    playlists = response['playlists']
    for i, item in enumerate(playlists['items']):
        print(playlists['offset'] + i, item['name'])

    if playlists['next']:
        response = sp.next(playlists)
    else:
        response = None

Editor's picks
0 New Music Friday
1 Feel Good Friday
2 RapCaviar
3 Main Stage
4 I Love My '90s Hip-Hop
5 Mood Booster
6 Dance Hits
7 Today's Top Hits
8 just hits
9 Dance Party
10 Happy 80s
11 young & free


The sixth result is called **"Mood Booster"**, which is highly related to my research objective, which is trying to analyse the mood of a song with sentimental analysis. I am curious about how the songs in this playlist are like.

To dig deeper into this playlist, I need to know the playlist's ID on Spotify. Unfortunately, the only feasible way to get a playlist's ID is through getting a user's current playlist. Here, I have to manually follow this playlist on Spotify and add it to my profile using my own Spotify account.

In [27]:
# Shows a user's playlists
import sys

if len(sys.argv) > 1:
    username = sys.argv[1]
else:
    print("Whoops, need a username!")
    print("usage: python user_playlists.py [username]")
    sys.exit()

# Using my own username as input
playlists = sp.user_playlists("214hpvbrd65hxnr2sq2rmwmxa")

for playlist in playlists['items']:
    print(playlist['id'],playlist['name'])


37i9dQZF1DX3rxVfibe1L0 Mood Booster


We now get the Spotify ID of the playlist. With this result as the the parameter of our next methods, we will be able to retrieve more information about this the playlist.

## 3. Tracks in the Playlist

We will utilise the Spotify **playlist_tracks** method to get a fixed output of the tracks in a certain playlist.

In [32]:
# Retrieves the track names in a playlist

# Input the playlist ID we got in the last step
pl_id = '37i9dQZF1DX3rxVfibe1L0'
offset = 0

while True:
    response = sp.playlist_items(pl_id,
                                 offset=offset,
                                 fields='items.track.name,total',
                                 additional_types=['track'])
    
    if len(response['items']) == 0:
        break
    
    pprint(response['items'])
    offset = offset + len(response['items'])

    print(offset, "/", response['total'])

[{'track': {'name': 'Little Bit of Love'}},
 {'track': {'name': 'Can I Get It'}},
 {'track': {'name': 'Dancing Feet (feat. DNCE)'}},
 {'track': {'name': 'Better Days (NEIKED x Mae Muller x Polo G)'}},
 {'track': {'name': "Let's Fall in Love for the Night"}},
 {'track': {'name': 'Meet Me At Our Spot'}},
 {'track': {'name': 'Heat Waves'}},
 {'track': {'name': 'When I’m Gone (with Katy Perry)'}},
 {'track': {'name': 'Glad You Exist'}},
 {'track': {'name': 'Wild (feat. Gary Clark Jr.)'}},
 {'track': {'name': 'Butterflies'}},
 {'track': {'name': 'Lil Bit'}},
 {'track': {'name': 'Overpass Graffiti'}},
 {'track': {'name': 'You (with Marshmello & Vance Joy)'}},
 {'track': {'name': 'The Bones - with Hozier'}},
 {'track': {'name': 'You Were Loved (with OneRepublic)'}},
 {'track': {'name': 'dancing in the kitchen'}},
 {'track': {'name': 'Love Again'}},
 {'track': {'name': 'Know Your Worth'}},
 {'track': {'name': 'Blueberry Eyes (feat. SUGA of BTS)'}},
 {'track': {'name': 'Heartbreak Anthem (with 

In [30]:
# Retrieves the track IDs in a playlist
from spotipy.oauth2 import SpotifyClientCredentials
import spotipy
from pprint import pprint


pl_id = '4xHnIEbBg8vyWLTiaT0kON'
offset = 0

while True:
    response = sp.playlist_items(pl_id,
                                 offset=offset,
                                 fields='items.track.id,total',
                                 additional_types=['track'])
    
    if len(response['items']) == 0:
        break
    
    pprint(response['items'])
    offset = offset + len(response['items'])

    print(offset, "/", response['total'])

HTTP Error for GET to https://api.spotify.com/v1/playlists/4xHnIEbBg8vyWLTiaT0kON/tracks with Params: {'limit': 100, 'offset': 0, 'fields': 'items.track.id,total', 'market': None, 'additional_types': 'track'} returned 404 due to Not found.


SpotifyException: http status: 404, code:-1 - https://api.spotify.com/v1/playlists/4xHnIEbBg8vyWLTiaT0kON/tracks?limit=100&offset=0&fields=items.track.id%2Ctotal&additional_types=track:
 Not found., reason: None

In [31]:
track = sp.track('2LwH6T39A5IODRgPv9XitR')
pprint(track)

{'album': {'album_type': 'single',
           'artists': [{'external_urls': {'spotify': 'https://open.spotify.com/artist/2ZmXexIJAD7PgABrj0qQRb'},
                        'href': 'https://api.spotify.com/v1/artists/2ZmXexIJAD7PgABrj0qQRb',
                        'id': '2ZmXexIJAD7PgABrj0qQRb',
                        'name': 'N.Flying',
                        'type': 'artist',
                        'uri': 'spotify:artist:2ZmXexIJAD7PgABrj0qQRb'}],
           'available_markets': ['AD',
                                 'AE',
                                 'AG',
                                 'AL',
                                 'AM',
                                 'AO',
                                 'AR',
                                 'AT',
                                 'AU',
                                 'AZ',
                                 'BA',
                                 'BB',
                                 'BD',
                                 'BE'

## References

Occhino, L., 2020. How to get your music featured on Spotify playlists. [online] Bandzoogle.com. Available at: <https://bandzoogle.com/blog/how-to-get-your-music-featured-on-spotify-playlists> [Accessed 8 April 2022].